# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 29 2022 10:13AM,247550,12,HH-37183,Hush Hush,Released
1,Sep 29 2022 10:13AM,247550,12,HH-37184,Hush Hush,Released
2,Sep 29 2022 10:13AM,247550,12,HH-37185,Hush Hush,Released
3,Sep 29 2022 10:13AM,247550,12,HH-37186,Hush Hush,Released
4,Sep 29 2022 10:13AM,247550,12,HH-37187,Hush Hush,Released
5,Sep 29 2022 10:13AM,247550,12,HH-37188,Hush Hush,Released
6,Sep 29 2022 10:13AM,247550,12,HH-37189,Hush Hush,Released
7,Sep 29 2022 10:13AM,247550,12,HH-37191,Hush Hush,Released
8,Sep 29 2022 10:12AM,247549,10,8587675,MedStone Pharma LLC,Released
9,Sep 29 2022 10:12AM,247549,10,8587676,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,247546,Released,1
25,247547,Released,1
26,247548,Released,1
27,247549,Released,11
28,247550,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247546,NaN,NaN,1.0
247547,NaN,NaN,1.0
247548,NaN,NaN,1.0
247549,NaN,NaN,11.0
247550,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247395,0.0,0.0,1.0
247419,0.0,0.0,1.0
247421,0.0,0.0,1.0
247471,0.0,0.0,1.0
247480,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247395,0,0,1
247419,0,0,1
247421,0,0,1
247471,0,0,1
247480,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247395,0,0,1
1,247419,0,0,1
2,247421,0,0,1
3,247471,0,0,1
4,247480,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247395,,,1
1,247419,,,1
2,247421,,,1
3,247471,,,1
4,247480,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 29 2022 10:13AM,247550,12,Hush Hush
8,Sep 29 2022 10:12AM,247549,10,MedStone Pharma LLC
19,Sep 29 2022 9:58AM,247548,10,Eywa Pharma Inc.
20,Sep 29 2022 9:55AM,247547,19,"NAPP Technologies, LLC"
21,Sep 29 2022 9:49AM,247546,19,"GUSA Granules USA, Inc."
22,Sep 29 2022 9:47AM,247543,12,LF of America Corp.
23,Sep 29 2022 9:44AM,247421,21,"NBTY Global, Inc."
24,Sep 29 2022 9:42AM,247419,21,"NBTY Global, Inc."
25,Sep 29 2022 9:26AM,247541,20,"Andelyn Biosciences, Inc."
26,Sep 29 2022 9:09AM,247537,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 29 2022 10:13AM,247550,12,Hush Hush,,,8
1,Sep 29 2022 10:12AM,247549,10,MedStone Pharma LLC,,,11
2,Sep 29 2022 9:58AM,247548,10,Eywa Pharma Inc.,,,1
3,Sep 29 2022 9:55AM,247547,19,"NAPP Technologies, LLC",,,1
4,Sep 29 2022 9:49AM,247546,19,"GUSA Granules USA, Inc.",,,1
5,Sep 29 2022 9:47AM,247543,12,LF of America Corp.,,,1
6,Sep 29 2022 9:44AM,247421,21,"NBTY Global, Inc.",,,1
7,Sep 29 2022 9:42AM,247419,21,"NBTY Global, Inc.",,,1
8,Sep 29 2022 9:26AM,247541,20,"Andelyn Biosciences, Inc.",,,1
9,Sep 29 2022 9:09AM,247537,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 10:13AM,247550,12,Hush Hush,8,,
1,Sep 29 2022 10:12AM,247549,10,MedStone Pharma LLC,11,,
2,Sep 29 2022 9:58AM,247548,10,Eywa Pharma Inc.,1,,
3,Sep 29 2022 9:55AM,247547,19,"NAPP Technologies, LLC",1,,
4,Sep 29 2022 9:49AM,247546,19,"GUSA Granules USA, Inc.",1,,
5,Sep 29 2022 9:47AM,247543,12,LF of America Corp.,1,,
6,Sep 29 2022 9:44AM,247421,21,"NBTY Global, Inc.",1,,
7,Sep 29 2022 9:42AM,247419,21,"NBTY Global, Inc.",1,,
8,Sep 29 2022 9:26AM,247541,20,"Andelyn Biosciences, Inc.",1,,
9,Sep 29 2022 9:09AM,247537,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 10:13AM,247550,12,Hush Hush,8,,
1,Sep 29 2022 10:12AM,247549,10,MedStone Pharma LLC,11,,
2,Sep 29 2022 9:58AM,247548,10,Eywa Pharma Inc.,1,,
3,Sep 29 2022 9:55AM,247547,19,"NAPP Technologies, LLC",1,,
4,Sep 29 2022 9:49AM,247546,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 10:13AM,247550,12,Hush Hush,8.0,NaN,NaN
1,Sep 29 2022 10:12AM,247549,10,MedStone Pharma LLC,11.0,NaN,NaN
2,Sep 29 2022 9:58AM,247548,10,Eywa Pharma Inc.,1.0,NaN,NaN
3,Sep 29 2022 9:55AM,247547,19,"NAPP Technologies, LLC",1.0,NaN,NaN
4,Sep 29 2022 9:49AM,247546,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 29 2022 10:13AM,247550,12,Hush Hush,8.0,0.0,0.0
1,Sep 29 2022 10:12AM,247549,10,MedStone Pharma LLC,11.0,0.0,0.0
2,Sep 29 2022 9:58AM,247548,10,Eywa Pharma Inc.,1.0,0.0,0.0
3,Sep 29 2022 9:55AM,247547,19,"NAPP Technologies, LLC",1.0,0.0,0.0
4,Sep 29 2022 9:49AM,247546,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2475267,44.0,38.0,10.0
12,495093,9.0,0.0,0.0
16,495014,2.0,0.0,0.0
19,742617,2.0,18.0,0.0
20,742594,4.0,0.0,0.0
21,1484710,5.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2475267,44.0,38.0,10.0
1,12,495093,9.0,0.0,0.0
2,16,495014,2.0,0.0,0.0
3,19,742617,2.0,18.0,0.0
4,20,742594,4.0,0.0,0.0
5,21,1484710,5.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,44.0,38.0,10.0
1,12,9.0,0.0,0.0
2,16,2.0,0.0,0.0
3,19,2.0,18.0,0.0
4,20,4.0,0.0,0.0
5,21,5.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,44.0
1,12,Released,9.0
2,16,Released,2.0
3,19,Released,2.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,0.0
Executing,38.0,0.0,0.0,18.0,0.0,1.0
Released,44.0,9.0,2.0,2.0,4.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,38.0,0.0,0.0,18.0,0.0,1.0
2,Released,44.0,9.0,2.0,2.0,4.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,38.0,0.0,0.0,18.0,0.0,1.0
2,Released,44.0,9.0,2.0,2.0,4.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()